In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType
from pyspark.sql.functions import udf, to_date, sum

In [11]:
spark.stop()
sc.stop()
conf = SparkConf().setAppName("Jupyter:\n GitHub")
conf = conf.setMaster("spark://david-hdp:7077")
conf = conf.set("spark.jars.packages", "com.databricks:spark-xml_2.11:0.4.1")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
spark

In [14]:
spark.read.format("xml").load("file.xml")

Py4JJavaError: An error occurred while calling o546.load.
: java.lang.ClassNotFoundException: Failed to find data source: xml. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:549)
	at org.apache.spark.sql.execution.datasources.DataSource.providingClass$lzycompute(DataSource.scala:86)
	at org.apache.spark.sql.execution.datasources.DataSource.providingClass(DataSource.scala:86)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:301)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:178)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:156)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Me

# Carga de datos y tablas 

In [5]:
errors = spark.read.json('git/git_errors.json')
errors = errors.filter(errors._corrupt_record.isNull()).drop(errors._corrupt_record)
errors.createOrReplaceTempView("errors")

langs = spark.read.json('git/git_langs.json')
langs.createOrReplaceTempView("langs")

commits = spark.read.json('git/git_commits.json')
commits.createOrReplaceTempView("commits")

### Creación de tablas temporales

In [4]:
errors.printSchema()
langs.printSchema()
commits.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nullable = true)
 |    |    |    |    |-- href

### Descubrir porque ha fallado

In [6]:
lang_error = spark.sql("SELECT repo.id, \
                                payload.pull_request.base.repo.language as repo_base_lang, \
                                payload.pull_request.head.repo as repo_head_lang \
                        FROM errors \
                        WHERE \
                            payload.pull_request.base.repo.language IS NOT NULL")

lang_error.createOrReplaceTempView("lang_error")

In [6]:
lang_error.show()
lang_error.count()

+--------+--------------+--------------+
|      id|repo_base_lang|repo_head_lang|
+--------+--------------+--------------+
|13746419|            C#|          null|
| 2464908|           PHP|          null|
|  117358|         Perl6|          null|
| 8869463|          Haxe|          null|
| 2625205|        Python|          null|
|10941409|    JavaScript|          null|
| 9426192|        Python|          null|
|23259263|    JavaScript|          null|
|  206341|          Java|          null|
| 1141571|    JavaScript|          null|
| 7850354|             C|          null|
|27539466|    JavaScript|          null|
| 1975671|           PHP|          null|
|17312031|    JavaScript|          null|
|10653734|        Scheme|          null|
|25968039|          Java|          null|
| 5782574|           Lua|          null|
|17293434|    JavaScript|          null|
| 5022564|    JavaScript|          null|
|  905738|   Objective-C|          null|
+--------+--------------+--------------+
only showing top

13673

El resultado debido del fallo es una modificación de la estructura del json.

En el **payload.pull_request.head.repo.language** la etiqueta _repo_ es un _String_ en vez de un _Struct_ que sería lo esperado.

La siguiente fase será recuperar estos datos.

### Compruebo que los esquemas de las fases que voy a unir sean iguales

In [7]:
lang_error.printSchema()
langs.printSchema()

root
 |-- id: long (nullable = true)
 |-- repo_base_lang: string (nullable = true)
 |-- repo_head_lang: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- repo_base_lang: string (nullable = true)
 |-- repo_head_lang: string (nullable = true)



### Selecciono los idiomas que no estan en la base original

In [7]:
lang_error = spark.sql("select * from lang_error where id not in (select id from langs)")

In [11]:
lang_error.count()

2024

### Uno los dos Dataframes

In [8]:
langs = langs.unionAll(lang_error)

In [10]:
langs.coalesce(1).write.json('langs_final')

### Extraigo las coincidencias asignandole a cada id los lenguages correspondientes

In [41]:
#Segunda fase
langs = spark.read.json('git/langs_final.json')
langs.createOrReplaceTempView("langs")

commits = spark.read.json('git/git_commits.json')
commits.createOrReplaceTempView("commits")

In [42]:
result = spark.sql("SELECT \
                        c.time, \
                        c.size, \
                        l.repo_base_lang, \
                        l.repo_head_lang \
                    FROM commits c \
                    INNER JOIN langs l ON c.id = l.id")

In [4]:
result.show()
result.printSchema()

+----------+----+--------------+--------------+
|      time|size|repo_base_lang|repo_head_lang|
+----------+----+--------------+--------------+
|2017-08-11|   1|          Java|          Java|
|2017-08-11|   1|          Java|          Java|
|2017-08-11|   1|          Java|          Java|
|2017-08-18|   1|          Java|          Java|
|2017-08-18|   1|          Java|          Java|
|2017-08-18|   1|          Java|          Java|
|2017-08-23|   1|          Java|          Java|
|2017-08-23|   1|          Java|          Java|
|2017-08-23|   1|          Java|          Java|
|2017-08-23|   3|          Java|          Java|
|2017-08-23|   3|          Java|          Java|
|2017-08-23|   3|          Java|          Java|
|2017-08-12|   2|        Python|        Python|
|2017-08-12|   2|        Python|        Python|
|2017-08-12|   2|        Python|        Python|
|2017-08-12|   3|        Python|        Python|
|2017-08-12|   3|        Python|        Python|
|2017-08-12|   3|        Python|        

In [43]:
def complete(x):
    if x[2] is None and x[3] is not None:
        return (x[0], x[1], x[3], x[3])
    return x

In [44]:
rstRDD = result.rdd.map(complete).filter(lambda x: x[2] is not None)

In [46]:
result = rstRDD.toDF()
result = result.groupBy(result.time, result.repo_base_lang, result.size).sum()

In [6]:
result.show()

+----------+----+--------------+--------------+
|      time|size|repo_base_lang|repo_head_lang|
+----------+----+--------------+--------------+
|2017-08-23|   1|          Java|          Java|
|2017-08-23|   1|          Java|          Java|
|2017-08-23|   1|          Java|          Java|
|2017-08-23|   3|          Java|          Java|
|2017-08-23|   3|          Java|          Java|
|2017-08-23|   3|          Java|          Java|
|2017-08-18|   1|          Java|          Java|
|2017-08-18|   1|          Java|          Java|
|2017-08-18|   1|          Java|          Java|
|2017-08-11|   1|          Java|          Java|
|2017-08-11|   1|          Java|          Java|
|2017-08-11|   1|          Java|          Java|
|2017-08-12|   2|        Python|        Python|
|2017-08-12|   2|        Python|        Python|
|2017-08-12|   2|        Python|        Python|
|2017-08-12|   3|        Python|        Python|
|2017-08-12|   3|        Python|        Python|
|2017-08-12|   3|        Python|        

# Export to Single CSV

In [47]:
sc.setJobGroup("Parket snappy", "Parket snappy save")
result.write.parquet('git/git_final_parquet',compression='snappy')

In [48]:
sc.setJobGroup("Json normal", "Json normal save")
result.write.json('git/git_final_normal')

In [ ]:
sc.setJobGroup("Json coalesce", "Json coalesce")
result.coalesce(1).write.csv('git_final_result', header=True)